In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import requests
import io
from PIL import Image
from time import sleep
import time
import random
import base64
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.common.exceptions import WebDriverException
import os
import pandas as pd
import numpy as np


In [2]:
#Crawl basic phone data
# Declare browser
s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

# Khởi tạo trình duyệt Chrome với tùy chọn toàn màn hình
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=s, options=chrome_options)

# Open URL
driver.get("https://www.thegioididong.com/adapter-sac-sac-xe-hoi#c=9499&p=182286,182285,237128,237127&o=14&pi=3")
sleep(random.randint(5,10))

# Scroll down
for i in range(5):
    scroll_script = "window.scrollBy(0, window.innerHeight);"
    try:
        driver.execute_script(scroll_script)
    except WebDriverException as e:
        print("WebDriverException:", e)
    time.sleep(1) 

# Scroll up
for i in range(20):
    scroll_script = "window.scrollBy(0, -window.innerHeight);"
    try:
        driver.execute_script(scroll_script)
    except WebDriverException as e:
        print("WebDriverException:", e)
    time.sleep(1)
    
# elems2 = driver.find_elements(By.CLASS_NAME , "jBwCF ")
# links = [elem.get_attribute('src') for elem in elems2 if elem.get_attribute('type') == 'product' and elem.get_attribute('src').count("data:image") <=0]

# ================================ GET id
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_9499 a')
id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]

# # ================================ GET title
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_9499 a')
title = [elem.get_attribute('data-name') for elem in elems if elem.get_attribute('data-name') is not None]

# ================================ GET img_thumb  .item.ajaxed.__cate_42
# elems = driver.find_elements(By.CLASS_NAME, 'thumb')
elems = driver.find_elements(By.CSS_SELECTOR, '.item-img.item-img_9499 img')
link_img_thumb = [elem.get_attribute('src') for elem in elems if elem.get_attribute('src') is not None and "Products" in elem.get_attribute('src')]

# ================================ GET href
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_9499 a')
href = [elem.get_attribute('href') for i, elem in enumerate(elems) if i % 3 == 0]

# ================================ GET price
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_9499 a')
price = [elem.get_attribute('data-price') for elem in elems if elem.get_attribute('data-price') is not None]

# Clean price
price = [int(float(number)) for number in price]

# ================================ GET price pre discount
pre_discount_list, discount_idx, discount_percent_list = [], [], []
for i in range(1, len(title)+1):
    try:
        discount_percent = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[3]/span".format(i))
        discount_percent_list.append(discount_percent.text)

        pre_discount = driver.find_element("xpath", "/html/body/div[6]/section/div[3]/ul/li[{}]/a[1]/div[3]/p".format(i))
        pre_discount_list.append(pre_discount.text)

        
        print(i)
        discount_idx.append(i)
    except NoSuchElementException:
        print("No Such Element Exception " + str(i))


# ================================ GET brand
elems = driver.find_elements(By.CSS_SELECTOR, '.item.__cate_9499 a')
brand = [elem.get_attribute('data-brand') for elem in elems if elem.get_attribute('data-brand') is not None]


# Close browser
driver.close() 

WebDriverException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=120.0.6099.110)
Stacktrace:
	GetHandleVerifier [0x00007FF6285382B2+55298]
	(No symbol) [0x00007FF6284A5E02]
	(No symbol) [0x00007FF6283605AB]
	(No symbol) [0x00007FF628340038]
	(No symbol) [0x00007FF6283C6BC7]
	(No symbol) [0x00007FF6283DA15F]
	(No symbol) [0x00007FF6283C1E83]
	(No symbol) [0x00007FF62839670A]
	(No symbol) [0x00007FF628397964]
	GetHandleVerifier [0x00007FF6288B0AAB+3694587]
	GetHandleVerifier [0x00007FF62890728E+4048862]
	GetHandleVerifier [0x00007FF6288FF173+4015811]
	GetHandleVerifier [0x00007FF6285D47D6+695590]
	(No symbol) [0x00007FF6284B0CE8]
	(No symbol) [0x00007FF6284ACF34]
	(No symbol) [0x00007FF6284AD062]
	(No symbol) [0x00007FF62849D3A3]
	BaseThreadInitThunk [0x00007FFE1362257D+29]
	RtlUserThreadStart [0x00007FFE1476AA58+40]

WebDriverException: Message: no such window: target window already closed
from unknown error: web vi

KeyboardInterrupt: 

In [3]:
#Clean discount_list
for i, number in enumerate(pre_discount_list):
    if pre_discount_list[i] and pre_discount_list[i] != '':
        if '.' in pre_discount_list[i]:
            pre_discount_list[i] = int(pre_discount_list[i].replace('.', '').replace('₫', ''))

In [4]:
#Merge & create excel
df1 = pd.DataFrame(list(zip(id_product, title, brand ,href ,link_img_thumb , price)), columns = ['id', 'title' ,'brand', 'href', 'link_img_thumb', 'price'])
# Đặt tên tệp Excel và sheet
excel_file = 'chargingadapter.xlsx'
sheet_name = 'Sheet1'

df1['index_']= np.arange(1, len(df1) + 1)

df2 = pd.DataFrame(list(zip(pre_discount_list ,discount_percent_list, discount_idx)), columns = ['discount', 'discount_percent','discount_idx'])
df3 = df1.merge(df2, how='left', left_on='index_', right_on='discount_idx')

# Lưu DataFrame vào tệp Excel
df3.to_excel(excel_file, sheet_name=sheet_name, index=False, engine='openpyxl')

In [5]:
len(link_img_thumb)

76

In [2]:
df = pd.read_excel('chargingadapter.xlsx')

link_img_thumb = df['link_img_thumb'].to_list()
href = df['href'].to_list()
id_product = df['id'].to_list()

print(href)

['https://www.thegioididong.com/adapter-sac/de-sac-khong-day-magsafe-3-in-1-15w-anker-737-cube-y1811', 'https://www.thegioididong.com/adapter-sac/adapter-sac-da-nang-universal-pd-65w-xmobile-ds636', 'https://www.thegioididong.com/adapter-sac/adapter-sac-2-cong-type-c-pd-gan-35w-mazer-gan35us', 'https://www.thegioididong.com/adapter-sac/adapter-sac-type-c-pd-gan5-20w-baseus-ccgan20c5-m', 'https://www.thegioididong.com/adapter-sac/adapter-sac-type-c-pd-gan-30w-xmobile-ds230-trang', 'https://www.thegioididong.com/adapter-sac/type-c-pd-25w-samsung-ep-ta800n', 'https://www.thegioididong.com/adapter-sac/adapter-sac-type-c-20w-cho-iphone-ipad-apple-mhje3', 'https://www.thegioididong.com/adapter-sac/adapter-sac-type-c-pd-20w-xmobile-a829-trang', 'https://www.thegioididong.com/adapter-sac/adapter-sac-type-c-pd-gan-30w-xmobile-mfm30-trang', 'https://www.thegioididong.com/adapter-sac/adapter-3-cong-usb-typecpd-kem-cap-typec-typec-1m-65w-baseus-ccgan65s5', 'https://www.thegioididong.com/adapter-sa

In [ ]:
def download_image(download_path, url, index):
    try:
        if url.startswith('data:image/'):
            # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
            image_data = url.split(',')[1]
            image_bytes = base64.b64decode(image_data)

            # Tạo một hình ảnh PIL từ các byte và lưu nó
            image = Image.open(io.BytesIO(image_bytes))
            # file_path = title[i] + file_name

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
        else:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            # file_path = title[i] + file_name
            
            file_path = os.path.join(download_path, f"{id_product[i]}.jpg")

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")

        print("Success ", index)
    except Exception as e:
        print('FAILED -', e)


urls = link_img_thumb

dowloadPath = r'D:\CodeUIT\Data\imgs\img_product'
for i, url in enumerate(urls):
	download_image(dowloadPath, url, i)

In [3]:
#Crawl parameters data
list_parameters=[]
for href_items in href:
# for i in range(1):

    # Declare browser
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    # Browser full window
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    #Open browser 
    driver.get(href_items)
    sleep(random.randint(1,2))

    # Scroll down
    for i in range(5):
        scroll_script = "window.scrollBy(0, window.innerHeight);"
        driver.execute_script(scroll_script)
        time.sleep(1) 

    elems = driver.find_elements(By.CSS_SELECTOR, '.parameter')
    # id_product = [elem.get_attribute('data-id') for elem in elems if elem.get_attribute('data-id') is not None]
    parameters = [elem.text for elem in elems]

    list_parameters.append(parameters[0])


    #Close browser
    driver.close()

In [5]:
list_parameters

['Model\nY1811\nChức năng\nSạc\nĐầu vào\nType-C\nĐầu ra\nMagSafe: 5W - 15W\nJack kết nối\nType-C\nDòng sạc tối đa\n15 W\nKích thước\nDài 6 cm - Ngang 6 cm - Cao 6.34 cm\nCông nghệ/Tiện ích\nSạc không dây\nSản xuất tại\nTrung Quốc\nThương hiệu của\nTrung Quốc\nHãng\nAnker. Xem thông tin hãng',
 'Model\nDS636\nChức năng\nSạc\nĐầu vào\n100-250V ~ 10A Max\nĐầu ra\nUSB A1/A2: 5V - 2.4A (Max 12W)\nType C1/C2: 5V - 3A (Max 15W)\nType C3: 5V - 3A, 9V - 3A, 12V - 3A, 15V - 3A, 20V - 3.25A (Max 65W)\nDòng sạc tối đa\n65 W\nKích thước\nDài 7.6 cm - Ngang 5.3 cm - Cao 8.8 cm\nCông nghệ/Tiện ích\nCông nghệ GaN\nPower Delivery\nBảo vệ khi nguồn điện quá 10 A\nSản xuất tại\nTrung Quốc\nThương hiệu của\nThế Giới Di Động\nHãng\nXmobile. Xem thông tin hãng',
 'Model\nGAN35US\nChức năng\nSạc\nĐầu vào\n100-240V~50/60Hz, 0.8A 100-240V ~ 1.5A 50-60Hz\nĐầu ra\nType C: 5V - 3A, 9V - 3A, 12V - 2.92A, 15V - 2.33A, 20V - 1.75A (Max 35W)\nDòng sạc tối đa\n35 W\nKích thước\nDài 4.9 cm - Ngang 4.9 cm - Cao 2.3 cm\n

In [76]:
model =[]
function =[]
input_ =[]
output =[]
maximum = []
size = []
tech = []
madein = []
brandof = [] 
brand = []
#Clean parameters data
for i, data in enumerate(list_parameters):
    data = data.replace("Xem thêm cấu hình chi tiết", "")
    # Tách dữ liệu thành danh sách các dòng
    data = data.replace('\n', ' ')
    # Tìm vị trí của các thông số trong dữ liệu
    if "Xem thông tin" in data:
        index = data.find(". Xem")
        data = data[0:index]
    if "Jack kết nối" in data:
        index_start = data.find("Jack kết nối")
        index_end = data.find("Dòng sạc")
        data = data[0:index_start-1] + data[index_end:]
    
    start_index = {
        "Model": data.find("Model"),
        "Chức năng": data.find("Chức năng"),
        "Đầu vào": data.find("Đầu vào"),
        "Đầu ra": data.find("Đầu ra"),
        "Dòng sạc tối đa": data.find("Dòng sạc tối đa"),
        "Kích thước": data.find("Kích thước"),
        "Công nghệ/Tiện ích": data.find("Công nghệ/Tiện ích"),
        "Sản xuất tại": data.find("Sản xuất tại"),
        "Thương hiệu của": data.find("Thương hiệu của"),
        "Hãng": data.rfind("Hãng")

    }

    # Tìm vị trí của các thông số kết thúc
    end_index = {
        "Model": data.find("Chức năng"),
        "Chức năng": data.find("Đầu vào"),
        "Đầu vào": data.find("Đầu ra"),
        "Đầu ra": data.find("Dòng sạc"),
        "Dòng sạc tối đa": data.find("Kích thước"),
        "Kích thước": data.find("Công nghệ"),
        "Công nghệ/Tiện ích": data.find("Sản xuất tại"),
        "Sản xuất tại": data.find("Thương hiệu của"),
        "Thương hiệu của": data.rfind("Hãng"),
        "Hãng": len(data)
    }

    # Khởi tạo một từ điển trống
    chargingadapter_info = {}

    # Duyệt qua các thông số để lấy giá trị
    for key in start_index.keys():
        if start_index[key] != -1 and end_index[key] != -1:
            value = data[start_index[key] + len(key + " "):end_index[key]]
            chargingadapter_info[key] = value

    if 'Model' in chargingadapter_info:
        model.append(chargingadapter_info['Model'])
    else:
        model.append(None)
    if 'Chức năng' in chargingadapter_info:
        function.append(chargingadapter_info['Chức năng'])
    else:
        function.append(None)
    if 'Đầu vào' in chargingadapter_info:
        input_.append(chargingadapter_info['Đầu vào'])
    else:
        input_.append(None) 
    if 'Đầu ra' in chargingadapter_info:
        output.append(chargingadapter_info['Đầu ra'])
    else:
        output.append(None) 
    if 'Dòng sạc tối đa' in chargingadapter_info:
        maximum.append(chargingadapter_info['Dòng sạc tối đa'])
    else:
        maximum.append(None) 
    if 'Kích thước' in chargingadapter_info:
        size.append(chargingadapter_info['Kích thước'])
    else:
        size.append(None) 
    if 'Công nghệ/Tiện ích' in chargingadapter_info:
        tech.append(chargingadapter_info['Công nghệ/Tiện ích'])
    else:
        tech.append(None)
    if 'Sản xuất tại' in chargingadapter_info:
        madein.append(chargingadapter_info['Sản xuất tại'])
    else:
        madein.append(None)
    if 'Thương hiệu của' in chargingadapter_info:
        brandof.append(chargingadapter_info['Thương hiệu của'])
    else:
        brandof.append(None)
    if 'Hãng' in chargingadapter_info:
        brand.append(chargingadapter_info['Hãng'])
    else:
        brand.append(None)
    print(chargingadapter_info)


{'Model': 'Y1811 ', 'Chức năng': 'Sạc ', 'Đầu vào': 'Type-C ', 'Đầu ra': 'MagSafe: 5W - 15W', 'Dòng sạc tối đa': '15 W ', 'Kích thước': 'Dài 6 cm - Ngang 6 cm - Cao 6.34 cm ', 'Công nghệ/Tiện ích': 'Sạc không dây ', 'Sản xuất tại': 'Trung Quốc ', 'Thương hiệu của': 'Trung Quốc ', 'Hãng': 'Anker'}
{'Model': 'DS636 ', 'Chức năng': 'Sạc ', 'Đầu vào': '100-250V ~ 10A Max ', 'Đầu ra': 'USB A1/A2: 5V - 2.4A (Max 12W) Type C1/C2: 5V - 3A (Max 15W) Type C3: 5V - 3A, 9V - 3A, 12V - 3A, 15V - 3A, 20V - 3.25A (Max 65W) ', 'Dòng sạc tối đa': '65 W ', 'Kích thước': 'Dài 7.6 cm - Ngang 5.3 cm - Cao 8.8 cm ', 'Công nghệ/Tiện ích': 'Công nghệ GaN Power Delivery Bảo vệ khi nguồn điện quá 10 A ', 'Sản xuất tại': 'Trung Quốc ', 'Thương hiệu của': 'Thế Giới Di Động ', 'Hãng': 'Xmobile'}
{'Model': 'GAN35US ', 'Chức năng': 'Sạc ', 'Đầu vào': '100-240V~50/60Hz, 0.8A 100-240V ~ 1.5A 50-60Hz ', 'Đầu ra': 'Type C: 5V - 3A, 9V - 3A, 12V - 2.92A, 15V - 2.33A, 20V - 1.75A (Max 35W) ', 'Dòng sạc tối đa': '35 W ', '

In [86]:
new_df2 = pd.DataFrame(list(zip(model, function, input_ ,output ,maximum, size, tech, madein, brandof, brand)), columns = ['model', 'function', 'input_', 'output', 'maximum', 'size', 'tech', 'madein', 'brandof', 'brand'])
new_df2.to_excel('charging_apdater_info_update.xlsx', sheet_name='Sheet 1', index=False, engine='openpyxl')

In [ ]:
brand

In [95]:
def download_image_details(download_path, url, index):
    try:
        if url.startswith('data:image/'):
            # Trích xuất dữ liệu hình ảnh được mã hóa Base64 từ data URI
            image_data = url.split(',')[1]
            image_bytes = base64.b64decode(image_data)

            # Tạo một hình ảnh PIL từ các byte và lưu nó
            image = Image.open(io.BytesIO(image_bytes))
            # file_path = title[i] + file_name

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")
        else:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            # file_path = title[i] + file_name
            
            file_path = os.path.join(download_path, f"{index}.jpg")

            with open(file_path, "wb") as f:
                image.save(f, "JPEG")

        print("Success ", index)
    except Exception as e:
        print('FAILED -', e)


# urls = link_img_thumb

# parent_folder = r'D:\CodeUIT\Data\imgs'
# folder_name = 'abc'

# folder_path = os.path.join(parent_folder, folder_name)

# os.makedirs(folder_path, exist_ok=True)

# dowloadPath = r'D:\CodeUIT\Data\imgs\img_product'
# for i, url in enumerate(urls):
# 	download_image(dowloadPath, url, i)

In [111]:
df = pd.read_excel('phoneupdate.xlsx')
href = df['href'].to_list()
id_product = df['id'].to_list()

In [117]:
href_test

['https://www.thegioididong.com/dtdd/iphone-15-pro-max-512gb',
 'https://www.thegioididong.com/dtdd/iphone-15-pro-max',
 'https://www.thegioididong.com/dtdd/iphone-15-pro-1tb',
 'https://www.thegioididong.com/dtdd/iphone-15-pro-512gb',
 'https://www.thegioididong.com/dtdd/iphone-15-pro-256gb',
 'https://www.thegioididong.com/dtdd/iphone-15-pro']

In [115]:
href_test = []
for i, data in enumerate(href):
    href_test.append(data)
    if i==5: break

In [88]:
len(href)

76

In [121]:
id_failed = []
id_success = []
for i,href_items in enumerate(href):
# for i in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    # driver.get("https://www.thegioididong.com/dtdd-apple-iphone")
    # driver.get("https://www.thegioididong.com/dtdd/samsung-galaxy-s23-fe")
    # driver.get("https://www.thegioididong.com/dtdd-samsung")
    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    for scroll_i in range(2):
        scroll_script = "window.scrollBy(0, window.innerHeight);"
        driver.execute_script(scroll_script)
        time.sleep(1) 

    # # Scroll up
    # for i in range(5):
    #     scroll_script = "window.scrollBy(0, -window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1)


    # elems = driver.find_elements(By.CSS_SELECTOR, '.parameter-item')
    # parameters_details = [elem.text for elem in elems]

    elems = driver.find_elements(By.CSS_SELECTOR, '.owl-stage .owl-item.active a img')
    pics_act = [elem.get_attribute('src') for elem in elems ]

    elems = driver.find_elements(By.CSS_SELECTOR, '.owl-stage .owl-item a img' )
    pics = [elem.get_attribute('data-src') for elem in elems  ]

    pics_act_clean = []
    pics_clean = []

    for data in pics_act:
        if data and isinstance(data, str) and 'Slider' in data:
            pics_act_clean.append(data)
    
    for data in pics:
        if data and isinstance(data, str) and 'Slider' in data:
            pics_clean.append(data)

    # print(pics_act_clean)
    # print(pics_clean)

    pics_act = pics_act_clean
    pics = pics_clean

    if pics_act and pics:
        pics.insert(0, pics_act[0])
    
        urls = pics

        parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        folder_name = str(id_product[i])

        folder_path = os.path.join(parent_folder, folder_name)

        os.makedirs(folder_path, exist_ok=True)

        dowloadPath = folder_path

        for z, url in enumerate(urls):
            download_image_details(dowloadPath, url, z)

        id_success.append(id_product[i])
        print('------Successful ', i)
    else:
        id_failed.append(id_product[i])


   
    driver.close()

Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
------Successful  0
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
------Successful  1
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  2
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  3
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  4
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
------Successful  5
FAILED - cannot write mode RGBA as JPEG
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
FAILED - cannot write mode RGBA as JPEG
Success  8
------Successfu

In [123]:
id_failed

[313194,
 302953,
 309835,
 314975,
 306994,
 317163,
 317164,
 317981,
 312966,
 275953,
 309864,
 314203,
 313202,
 313199,
 316771,
 317221]

In [ ]:
i=0
for data in pics_clean:
    print(data, '\n')
    i+=1
    

print(i)

In [74]:
id_success

[]

In [45]:
i=0
for data in pics_act:
    if data is not None and '305659' in data and 'thumb' not in data:
        print(data, '\n')
        i+=1
print(i)
# for data in pics_act:
#     if data is not None :
#         print(data, '\n')
print(pics_act)

0
[]


In [43]:
df = pd.read_excel('phoneupdate.xlsx')
href = df['href'].to_list()
id_product = df['id'].to_list()
print(id_product)

[305659, 305658, 303833, 303832, 303831, 299033, 303825, 303823, 303891, 303812, 303716, 281570, 307245, 251192, 309814, 305695, 306980, 306979, 285082, 283828, 309816, 309846, 299734, 303163, 234621, 306785, 305887, 301603, 292672, 311355, 310805, 309746, 289705, 301608, 301796, 289702, 317163, 317164, 289694, 249948, 301798, 309745, 289700, 285031, 290829, 289691, 299250, 247508, 289710, 250625, 264060, 289663, 245545, 250258, 250622, 240259, 299034, 235838, 223602, 228736, 285696, 260546, 213031, 309821, 258047, 210644, 267212, 315401, 153856, 304222, 314358, 267211, 282389, 309722, 250103, 303585, 301642, 302531, 278886, 303179, 307891, 297026, 247361, 298377, 301641, 309834, 271717, 306774, 303583, 299248, 283148, 304162, 309847, 310647, 307203, 305886, 311120, 273335, 274018, 311354, 304090, 292780, 303298, 285224, 275435, 286697, 271721, 317530, 292770, 291696, 313154, 304182, 315651, 317974, 313320, 303579, 313153, 307556, 288401, 291697, 283819, 291625, 288404, 304276, 309723,

In [98]:
color_item = []
for index, href_items in enumerate(href):
# for index in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    # for scroll_i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 
    print(id_product[index])

    # elems = driver.find_elements(By.CSS_SELECTOR, '.box03.color.group.desk a')
    # list_color = [elem.text for elem in elems if elem.text is not None]

    # len_color = len(elems)
    # len_loop = len_color+2
    
    elems_loop = driver.find_elements(By.CSS_SELECTOR, '.scrolling_inner .box01__tab.scrolling .item.itemTab')
    loop = [elem_loop for elem_loop in elems_loop]
    list_color = []

    for i in range(1,len(loop)):
        elem_find = driver.find_element(By.XPATH, "/html/body/section[1]/div[3]/div[1]/div[1]/div[2]/div/div[{}]".format(i))
        elem_find.click()

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active')
        check = [elem.get_attribute('data-gallery-id') for elem in elems if elem.get_attribute('data-gallery-id') is not None]

        if check[0] != 'color-images-gallery':
            continue

        sleep(2)

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active .item-border img')
        color = [elem.get_attribute('alt') for elem in elems if elem.get_attribute('alt') is not None]
        list_color.append(color[0])

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active .item-border img')
        thumb_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')
        href_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]
        

        # for item in href_color:
        #     if item is None or str(id_product[index]) not in item:
        #         href_color.remove(item)

        # urls = href_color

        # parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        # folder_name = str(id_product[index])
        # folder_path = os.path.join(parent_folder, folder_name, color[0])

        # os.makedirs(folder_path, exist_ok=True)

        # dowloadPath = folder_path
        
        # thumb_color[0] = 'https:' + thumb_color[0]
        # download_image_details(dowloadPath, thumb_color[0], 'thumb')

        # for z, url in enumerate(urls):
        #     url = 'https:' + url
        #     download_image_details(dowloadPath, url, z)
    if len(list_color) >1 :
        color_item.append(', '.join(list_color))
    else: color_item.append(list_color[0])
    
    driver.close()

315355
314633
313911
310565
296250
234361
230315
275456
264973
310755
310634
310612
308527
271812
271813
232644
298454
232817
286272
314391
312255
310757
310756
310754
310641
310633
310618
305988
304046
292276
284547
284426
274201
274198
272201
296766
269385
266283
264378
248337
248329
247334
246060
245601
245600
241288
240627
236577
236575
232816
232815
232814
232645
232642
232422
231084
231083
230266
229595
227068
227067
225607
224746
218375
216063
211647
210429
203248
201510
201509
201112
197293
197291
135346
135345
100428


In [99]:
color_item

['Đen, Đen',
 'Đen, Đen',
 'Trắng, Đen, Đen',
 'Đen - Cam, Đen - Cam',
 'Trắng, Trắng',
 'Trắng, Đen, Đen',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Đen, Đen',
 'Đen, Đen',
 'Đen, Đen',
 'Đen, Đen',
 'Đen, Đen',
 'Đen, Đen',
 'Trắng, Trắng',
 'Xanh da trời, Tím, Xanh mint, Trắng, Trắng',
 'Hồng, Hồng',
 'Trắng, Trắng',
 'Xám, Xám',
 'Đen, Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Đen, Đen',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Đen, Đen',
 'Đen, Đen',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Xanh mint, Xanh mint',
 'Đen, Đen',
 'Đen, Đen',
 'Trắng, Trắng',
 'Đen, Trắng, Trắng',
 'Đen, Đen',
 'Trắng, Trắng',
 'Đen, Đen',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Đen, Đen',
 'Trắng, Đen, Đen',
 'Trắng, Đen, Đen',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Xanh Navy, Xanh Navy',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Trắng',
 'Trắng, Đen, Đen',
 'Trắng, Trắng',
 'Đen, Trắng, Trắng',
 'T

In [107]:
for index, href_items in enumerate(href_failed):
# for index in range(1):
    s = Service(r"D:\CodeUIT\Data\Crawl-Data-Phone-Shop\chromedriver.exe")

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=s, options=chrome_options)

    driver.get(href_items)
    # sleep(random.randint(1,2))
    sleep(0.5)
    # Scroll down
    # for scroll_i in range(2):
    #     scroll_script = "window.scrollBy(0, window.innerHeight);"
    #     driver.execute_script(scroll_script)
    #     time.sleep(1) 
    print(id_failed[index])

    elems = driver.find_elements(By.CSS_SELECTOR, '.box03.color.group.desk a')
    list_color = [elem.text for elem in elems if elem.text is not None]

    len_color = 1
    if len(elems) != 0:
        len_color = len(elems) 
    
    len_loop = len_color+2
    
    i = 1 
    while i < len_loop:
        elem_find = driver.find_element(By.XPATH, "/html/body/section[1]/div[3]/div[1]/div[1]/div[2]/div/div[{}]".format(i))
        elem_find.click()

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active')
        check = [elem.get_attribute('data-gallery-id') for elem in elems if elem.get_attribute('data-gallery-id') is not None]

        if len(check) >0 :
            if check[0] != 'color-images-gallery' and check[0]:
                print(check)
                len_loop += 1
                i+=1
                continue
            else:
                i += 1
        else: break

        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')))
        sleep(2)

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active .item-border img')
        color = [elem.get_attribute('alt') for elem in elems if elem.get_attribute('alt') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.item.itemTab.active-scroll-to.active .item-border img')
        thumb_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]

        elems = driver.find_elements(By.CSS_SELECTOR, '.show-tab.active .owl-stage .owl-item img')
        href_color = [elem.get_attribute('data-src') for elem in elems if elem.get_attribute('data-src') is not None]
        

        for item in href_color:
            if item is None or str(id_failed[index]) not in item:
                href_color.remove(item)

        urls = href_color

        parent_folder = r'D:\CodeUIT\Data\imgs\imgs_details'
        folder_name = str(id_failed[index])
        folder_path = os.path.join(parent_folder, folder_name, color[0])

        os.makedirs(folder_path, exist_ok=True)

        dowloadPath = folder_path
        
        download_image_details(dowloadPath, thumb_color[0], 'thumb')

        for z, url in enumerate(urls):
            url = 'https:' + url
            download_image_details(dowloadPath, url, z)
    # color_item.append(', '.join(list_color))
    color_item[index_failed[index]] = color[0]
        

    driver.close()

305695
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
317163
Success  thumb
Success  0
Success  1
Success  2
Success  3
FAILED - cannot write mode P as JPEG
317164
Success  thumb
Success  0
Success  1
Success  2
Success  3
FAILED - cannot write mode P as JPEG
289694
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
250622
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
285696
['featured-images-gallery']
Success  thumb
Success  0
Success  1
Success  2
Success  3
Success  4
Success  5
Success  6
Success  7
Success  8
Success  9
Success  10
Success  11
Success  12
Success  13
Success  14
Success  15
309821
['featured-images-gallery']
Succes

In [105]:
index_failed = []
for it in id_failed:
    for x, item in enumerate(id_product):
        if it == item:
            index_failed.append(x)

In [109]:
new_df2 = pd.DataFrame(list(zip(color_item)), columns = ['color_item'])
new_df2.to_excel('color_item_adapter.xlsx', sheet_name='Sheet 1', index=False, engine='openpyxl')

In [108]:
color_item

['Titan xanh, Titan đen, Titan tự nhiên, Titan trắng',
 'Titan trắng, Titan đen',
 'Titan tự nhiên, Titan đen, Titan xanh, Titan trắng',
 'Titan đen, Titan tự nhiên, Titan trắng, Titan xanh',
 'Titan đen, Titan xanh, Titan tự nhiên, Titan trắng',
 'Titan tự nhiên, Titan xanh, Titan trắng, Titan đen',
 'Xanh lá nhạt, Đen, Xanh dương nhạt, Hồng nhạt, Vàng nhạt',
 'Đen, Xanh dương nhạt, Vàng nhạt, Hồng nhạt, Xanh lá nhạt',
 'Xanh dương nhạt, Đen, Hồng nhạt, Vàng nhạt, Xanh lá nhạt',
 'Xanh dương nhạt, Xanh lá nhạt, Hồng nhạt, Đen, Vàng nhạt',
 'Hồng nhạt, Vàng nhạt, Xanh lá nhạt, Xanh dương nhạt, Đen',
 'Hồng nhạt, Xanh dương nhạt, Đen, Vàng nhạt, Xanh lá nhạt',
 'Xanh Dương, Bạc, Đen',
 'Tím, Đen, Vàng',
 'Xanh Dương, Xanh lá, Đen',
 'Xanh Dương',
 'Tím, Xám',
 'Xám, Tím',
 'Đen, Xanh lá, Vàng',
 'Đen, Vàng',
 'Xanh lá, Xanh Dương, Đen',
 'Vàng, Xanh lá, Xám, Xanh Dương',
 'Xanh dương nhạt, Xám, Xanh Dương',
 'Xám, Xanh Dương, Xanh lá',
 'Xám, Xanh Dương, Tím',
 'Vàng, Đen',
 'Xanh Dương